In [4]:
import pandas as pd
import time
from collections import Counter
from nltk import word_tokenize 
# https://www.nltk.org/_modules/nltk/tokenize.html
# Tokenizers divide strings into lists of substrings.  For example,
# tokenizers can be used to find the words and punctuation in a string
import nltk
import numpy as np
#from wordcloud import WordCloud, STOPWORDS # STOPWORDS
from nltk.corpus import stopwords # stopwords 
import matplotlib.pyplot as plt
import seaborn as sns
import string
import pickle

In [8]:
def remove_values_from_list(the_list, val):
    return [value for value in the_list if value != val]

def my_preprocess(sentence):
    sentence = str(sentence)
    sentence = sentence.lower() # Step 1: transfer data to lower-case strings
    list_words = word_tokenize(sentence) #Step 2: break the sentences into a list of words
    list_words = remove_values_from_list(list_words, '$') # Step 3: $hashtag$ and $url$ must be filtered
    list_words = remove_values_from_list(list_words, 'hashtag')
    list_words = remove_values_from_list(list_words, 'url')
    # Step 4: filter punctuations
    list_words_return = []
    for s in list_words:
        if s not in string.punctuation:
            list_words_return.append(s)
    # Step 5: some rows might disapear, I give " " to them
    if len(list_words_return) == 0:
        list_words_return.append(" ")
    return (' '.join(list_words_return)) # return a string which words seperated by ' '

def my_preprocess2(sentence):
    sentence = str(sentence)
    sentence = sentence.lower() # Step 1: transfer data to lower-case strings
    list_words = word_tokenize(sentence) #Step 2: break the sentences into a list of words
    list_words = remove_values_from_list(list_words, '$') # Step 3: $hashtag$ and $url$ must be filtered
    list_words = remove_values_from_list(list_words, 'hashtag')
    list_words = remove_values_from_list(list_words, 'url')
    # Step 4: filter punctuations
    list_words_return = []
    for s in list_words:
        if s not in string.punctuation:
            list_words_return.append(s)
    # Step 5: some rows might disapear, I give " " to them
    if len(list_words_return) == 0:
        list_words_return.append(" ")
    return (list_words_return) # return a list of words

# Section A: read in data and do filtering by punctuations, lower-case transfering, numbers

In [7]:
# read in train data
start_time = time.time()
path = "/mnt/c/Users/Administrator.DESKTOP-OHBO1TS/Desktop/Prof_Eesley/github/"
df = pd.read_excel(path + "training_set_dec9.xlsx")
print("df.shape",df.shape)
print("===== before fitering =====")
print(df['full_text'][0])
print("this cell takes time:",round(time.time() - start_time,3),"s")

df.shape (10381, 12)
===== before fitering =====
get started with $HASHTAG$ - check out $URL$ and install the client at: $URL$
this cell takes time: 1.612 s


In [9]:
# filtering data
start_time = time.time() 
inputDF = df['full_text'].apply(lambda x: my_preprocess(x) )
inputDF = pd.Series(inputDF)
print("I have filtered $hashtag$ and $url$")
print("I have filtered all punctuations, these are:",string.punctuation)
print("===== after fitering =====")
print("each row has datatype= ",type(inputDF[0]))
print(inputDF[0])
print("this cell takes time:",round(time.time() - start_time,3),"s")

I have filtered $hashtag$ and $url$
I have filtered all punctuations, these are: !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
===== after fitering =====
each row has datatype=  <class 'str'>
get started with check out and install the client at
this cell takes time: 4.257 s


In [10]:
start_time = time.time()
inputDF_words = df['full_text'].apply(lambda x: my_preprocess2(x) )
print("this cell takes time:",round(time.time() - start_time,3),"s")

this cell takes time: 3.686 s


In [100]:
len(inputDF_words)

10381

In [11]:
# remove punctuation and number/digits
start_time = time.time()

bag_of_words = []
for el in inputDF_words:
    bag_of_words.extend(el)
bag_of_words = list(np.unique(bag_of_words))

print("There are ",len(bag_of_words)," unique words in bag_of_words")
bag_of_words.sort()

starting_words = list(string.ascii_lowercase[:26]) # a-z
ban_words = [str(i) for i in range(0,10)] # 0-9
ban_punctuation = list(string.punctuation)

bag_of_words_new = []
for i in range(len(bag_of_words)):
    #if (i % 100 ==0):
        #print("dealing with word count = ",i)
    words = bag_of_words[i]
    flag1 = True
    flag2 = False
    flag3 = True
    
    for num in ban_words:
        if num in words:
            flag1 = False
            break
    
    for punct in ban_punctuation:
        if punct in words:
            flag3 = False
            break
    
    if words[0] in starting_words:
        flag2 = True
    
    if flag1 and flag2 and flag3:
        bag_of_words_new.append(words)
    else:
        continue
bag_of_words_new = list(np.unique(bag_of_words_new))

print("After filtering, there are ",len(bag_of_words_new)," unique words in bag_of_words")
print("this cell takes time:",round(time.time() - start_time,3),"s")

There are  12225  unique words in bag_of_words
After filtering, there are  10209  unique words in bag_of_words
this cell takes time: 0.338 s


In [15]:
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/zhiyuan8/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


AttributeError: 'set' object has no attribute 'words'

In [13]:
bag_of_words_new = [word for word in bag_of_words_new if word not in stopwords] # filter by nltk

bag_of_words_new = list(np.unique(bag_of_words_new))
print("After filtering, there are ",len(bag_of_words_new)," unique words in bag_of_words_new2")

After filtering, there are  10069  unique words in bag_of_words_new2


In [18]:
# save to file before stemming
f = open(path +  'Zhiyuan_Li_word_dict_before_stemming.csv','w+')
for t in bag_of_words_new:
    f.write(t + '\n')
f.close()
print("finished saving file")

finished saving file


# Section B: check spelling error 
https://rustyonrampage.github.io/text-mining/2017/11/28/spelling-correction-with-python-and-nltk.html

#### English words have a max of two repeated characters. Additional characters need to ripped off, otherwise we might add misleading information.

In [36]:
import re

def reduce_lengthening(text):
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)

In [37]:
reduce_lengthening( "aaand" )

'aand'

#### NLTK has no spell correction module, but there are many other libraries that can perform this task. I’ll using pattern en for this purpose

In [34]:
from spellchecker import SpellChecker

In [38]:
spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown(['aand','abandon','abandoned','abandonment','abc','abe','abi','ability','able','aboard',\
                            'abolition','abra','abroad','abruptly','absent','absolute','absolutely','abstract',\
                            'abstracted','abstraction'])

for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(spell.candidates(word))

aura
{'abba', 'bra', 'azra', 'abrc', 'abta', 'sabra', 'cabra', 'agra', 'absa', 'bara', 'aba', 'abram', 'aura', 'abri', 'apra', 'ambra', 'ara'}
and
{'wand', 'fand', 'rand', 'hand', 'anand', 'amand', 'sand', 'and', 'land', 'band'}


## B-main: run spelling check and build a map between wrong and corrected words

In [70]:
print("before filtering, number of words:",len(bag_of_words_new))

before filtering, number of words: 10069


In [71]:
bag_of_words_dict_1 = dict()

# round 1: delete trible duplicating letters 
for word in bag_of_words_new:
    bag_of_words_dict_1[word] = reduce_lengthening(word)
    if word != reduce_lengthening(word):
        print(word," trimmed to be: ",reduce_lengthening(word))

aaand  trimmed to be:  aand
aliiiiveee  trimmed to be:  aliivee
baaaaad  trimmed to be:  baad
dundunduuun  trimmed to be:  dundunduun
duuun  trimmed to be:  duun
fiiiiire  trimmed to be:  fiire
hmmm  trimmed to be:  hmm
huuuuuge  trimmed to be:  huuge
ieee  trimmed to be:  iee
iii  trimmed to be:  ii
litttle  trimmed to be:  little
lll  trimmed to be:  ll
mmmhmm  trimmed to be:  mmhmm
mooooooooon  trimmed to be:  moon
pssst  trimmed to be:  psst
purrrduce  trimmed to be:  purrduce
shhh  trimmed to be:  shh
waaay  trimmed to be:  waay


In [67]:
# round 2: correct the misspelling words
start_time = time.time()
spell = SpellChecker()

misspelled = spell.unknown( list(bag_of_words_dict_1.values()) ) # use trible duplicate deleted words

for word in misspelled:
    # Get the one `most likely` answer
    print(word," is corrected to be: ",spell.correction(word))

grav  is corrected to be:  grave
firefox  is corrected to be:  firebox
vinay  is corrected to be:  vinny
airdropping  is corrected to be:  airdropping
admins  is corrected to be:  admits
toothfairy  is corrected to be:  toothfairy
raincheck  is corrected to be:  raincheck
seraphid  is corrected to be:  seraphic
phecda  is corrected to be:  theda
refactoring  is corrected to be:  factoring
duh  is corrected to be:  due
coingate  is corrected to be:  coinage
carblock  is corrected to be:  warlock
dvlpmt  is corrected to be:  dvlpmt
abra  is corrected to be:  aura
stocklands  is corrected to be:  docklands
transacts  is corrected to be:  transact
binance  is corrected to be:  finance
swissnex  is corrected to be:  swissnex
validator  is corrected to be:  validators
changsha  is corrected to be:  changsha
gettin  is corrected to be:  getting
hashrate  is corrected to be:  castrate
walkthough  is corrected to be:  although
innoxneo  is corrected to be:  innoxneo
tba  is corrected to be:  te

hotstuff  is corrected to be:  hotstuff
unconventionnal  is corrected to be:  unconventional
junyu  is corrected to be:  june
guarda  is corrected to be:  guard
dyverse  is corrected to be:  diverse
gaians  is corrected to be:  gains
cafebar  is corrected to be:  caesar
oancea  is corrected to be:  once
tfw  is corrected to be:  tow
parroting  is corrected to be:  parting
iring  is corrected to be:  bring
zielin  is corrected to be:  violin
chadha  is corrected to be:  chad
evotion  is corrected to be:  emotion
liqu  is corrected to be:  liu
hotfix  is corrected to be:  hotfix
intrade  is corrected to be:  intrude
anarchapulco  is corrected to be:  anarchapulco
hiroto  is corrected to be:  hirst
spcc  is corrected to be:  nspcc
igital  is corrected to be:  digital
artmanov  is corrected to be:  romanov
chainhopping  is corrected to be:  chainhopping
ands  is corrected to be:  and
clowder  is corrected to be:  clower
artamonov  is corrected to be:  artamonov
botnets  is corrected to be:

usdx  is corrected to be:  us
utrust  is corrected to be:  trust
asos  is corrected to be:  asks
husian  is corrected to be:  human
bitcoine  is corrected to be:  bitching
mainet  is corrected to be:  magnet
swft  is corrected to be:  soft
vertbase  is corrected to be:  verbose
fadada  is corrected to be:  canada
iohk  is corrected to be:  work
owallet  is corrected to be:  wallet
dgamemaker  is corrected to be:  framemaker
inu  is corrected to be:  in
erts  is corrected to be:  arts
arkansians  is corrected to be:  arkansas
liana  is corrected to be:  diana
blockasset  is corrected to be:  blockasset
elastos  is corrected to be:  elastic
plugins  is corrected to be:  plains
redecentralize  is corrected to be:  decentralize
gamed  is corrected to be:  game
scrypt  is corrected to be:  script
monkeycoin  is corrected to be:  monkeycoin
motorspeedway  is corrected to be:  motorspeedway
karapetsas  is corrected to be:  karapetsas
akomba  is corrected to be:  aroma
truebit  is corrected to

minsauce  is corrected to be:  minsauce
dogeillionaire  is corrected to be:  dogeillionaire
newsbtc  is corrected to be:  newsboy
schnorr  is corrected to be:  schnur
weifund  is corrected to be:  refund
maian  is corrected to be:  main
favorites  is corrected to be:  favourites
tsankov  is corrected to be:  tanko
coursera  is corrected to be:  course
mintchalk  is corrected to be:  mintchalk
cryptocribs  is corrected to be:  cryptocribs
lixar  is corrected to be:  liar
cloudflare  is corrected to be:  cloudflare
bitregistro  is corrected to be:  bitregistro
leverington  is corrected to be:  everington
annouce  is corrected to be:  announce
eid  is corrected to be:  did
hexlant  is corrected to be:  extant
jentzsch  is corrected to be:  jentzsch
qryptos  is corrected to be:  crypts
krw  is corrected to be:  kew
retweeted  is corrected to be:  retreated
protocolo  is corrected to be:  protocol
colourise  is corrected to be:  colourist
simplepolicy  is corrected to be:  simplepolicy
cert

kappler  is corrected to be:  apple
webhallen  is corrected to be:  webhallen
reactjs  is corrected to be:  reacts
olabilir  is corrected to be:  olabilir
oyente  is corrected to be:  odette
solium  is corrected to be:  sodium
hangout  is corrected to be:  handout
bakon  is corrected to be:  bacon
sundog  is corrected to be:  sunday
yunbi  is corrected to be:  fungi
attwater  is corrected to be:  atwater
neoland  is corrected to be:  newland
reupload  is corrected to be:  reload
jk  is corrected to be:  uk
bolchevshy  is corrected to be:  bolchevshy
bontje  is corrected to be:  bottle
chainhops  is corrected to be:  chainhops
sparkpool  is corrected to be:  sparkpool
github  is corrected to be:  citub
interns  is corrected to be:  intern
uploaded  is corrected to be:  unloaded
yonsei  is corrected to be:  onset
rizun  is corrected to be:  run
lohja  is corrected to be:  lorna
eur  is corrected to be:  our
hexa  is corrected to be:  hela
rebating  is corrected to be:  relating
apac  is 

verizon  is corrected to be:  version
snapchat  is corrected to be:  snapshot
undeterminism  is corrected to be:  indeterminism
coinmex  is corrected to be:  coined
urls  is corrected to be:  curls
permissionlessness  is corrected to be:  permissionlessness
cointext  is corrected to be:  context
bots  is corrected to be:  both
sorries  is corrected to be:  worries
cryptochicks  is corrected to be:  cryptochicks
sayin  is corrected to be:  saying
mandeleil  is corrected to be:  mandeleil
mainnets  is corrected to be:  manners
wechat  is corrected to be:  what
sperb  is corrected to be:  superb
paradex  is corrected to be:  parade
sdks  is corrected to be:  sds
swoosh  is corrected to be:  woosh
huobi  is corrected to be:  hub
rescan  is corrected to be:  rescue
yann  is corrected to be:  ann
prelaunch  is corrected to be:  relaunch
hodled  is corrected to be:  howled
zenginli  is corrected to be:  zenginli
gamer  is corrected to be:  game
olofsson  is corrected to be:  olsson
antivirus 

marketsite  is corrected to be:  marketsite
nother  is corrected to be:  other
poolaid  is corrected to be:  poland
collboration  is corrected to be:  collaboration
codebases  is corrected to be:  codebases
fungibles  is corrected to be:  fungible
reitwiessner  is corrected to be:  reitwiessner
devgrants  is corrected to be:  deviants
assett  is corrected to be:  assets
surveils  is corrected to be:  surveys
eufemio  is corrected to be:  eufemia
cowgirls  is corrected to be:  howgills
otoh  is corrected to be:  ooh
roadmpas  is corrected to be:  roadmap
sande  is corrected to be:  sand
https  is corrected to be:  steps
ubitus  is corrected to be:  cubitus
imandra  is corrected to be:  sandra
ectous  is corrected to be:  sectors
trollery  is corrected to be:  trolley
chickun  is corrected to be:  chicken
cneo  is corrected to be:  cleo
lefteris  is corrected to be:  letters
namecoin  is corrected to be:  namecoin
plasmachain  is corrected to be:  plasmachain
quickstart  is corrected to 

bledel  is corrected to be:  level
sikorka  is corrected to be:  sikora
kansans  is corrected to be:  kansas
inbetween  is corrected to be:  in between
toolchain  is corrected to be:  toolchain
cryptosphere  is corrected to be:  cryptosphere
uajs  is corrected to be:  uaps
dji  is corrected to be:  di
cto  is corrected to be:  to
wrkshp  is corrected to be:  workshop
aplicaciones  is corrected to be:  aplicaciones
seo  is corrected to be:  so
async  is corrected to be:  sync
financieras  is corrected to be:  financiers
realcoin  is corrected to be:  realcoin
honglei  is corrected to be:  longley
trustless  is corrected to be:  trustees
ontrade  is corrected to be:  trade
dvp  is corrected to be:  dip
spass  is corrected to be:  pass
illinoisans  is corrected to be:  illinoisans
schoedon  is corrected to be:  schoon
freeh  is corrected to be:  free
ubuntu  is corrected to be:  bantu
chikara  is corrected to be:  chiara
firewalled  is corrected to be:  firewalled
originalmy  is corrected

progpow  is corrected to be:  progpow
oklahomans  is corrected to be:  oklahoma
avg  is corrected to be:  ag
gdax  is corrected to be:  gda
initializing  is corrected to be:  initializing
beauregard  is corrected to be:  beauregard
fud  is corrected to be:  fund
dappzone  is corrected to be:  dappzone
localbitcoins  is corrected to be:  localbitcoins
whops  is corrected to be:  shops
kbgs  is corrected to be:  bgs
yasen  is corrected to be:  rasen
sia  is corrected to be:  six
qihoo  is corrected to be:  hoo
zug  is corrected to be:  dug
btc  is corrected to be:  etc
bitcoinxt  is corrected to be:  bitcoinxt
tumblr  is corrected to be:  tumble
youtubers  is corrected to be:  youtubers
vdfs  is corrected to be:  vdus
diffie  is corrected to be:  diffi
yakobovitch  is corrected to be:  yakobovitch
jae  is corrected to be:  joe
pix  is corrected to be:  six
ethereumclassicnetwork  is corrected to be:  ethereumclassicnetwork
baad  is corrected to be:  bad
decal  is corrected to be:  deal
e

#### after browsing the modification list, I manually decide these spelling checks are valid:
aand is corrected to be:  and  
gettin	is	corrected	to	be:	getting  
yez is	corrected	to	be:	yes
aliivee is	corrected	to	be:	alive  
fiire  is	corrected	to	be:	fire  
huuge is	corrected	to	be:	huge  
waay  is	corrected	to	be:	way

In [79]:
# modify bag_of_words_dict_1

transfer_dict = {'aand':'and','gettin':'getting','yez':'yes','fiire':'fire','huuge':'huge','waay':'way'}
for key,value in bag_of_words_dict_1.items():
    if value in list(transfer_dict.keys()):
        bag_of_words_dict_1[key] = transfer_dict[value]
        print(key,"->",transfer_dict[value])
        

fiiiiire -> fire
gettin -> getting
huuuuuge -> huge
waaay -> way
yez -> yes


# Section C: for 10069 words, do stemming
https://www.datacamp.com/community/tutorials/stemming-lemmatization-python  
Stemming is the process of reducing inflection in words to their root forms such as mapping a group of words to the same stem even if the stem itself is not a valid word in the Language.

In [16]:
nltk.download()

NLTK Downloader
---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> punkt
Command 'punkt' unrecognized

---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> d

Download which package (l=list; x=cancel)?
  Identifier> punkt


      Package punkt is already up-to-date!



---------------------------------------------------------------------------
    d) Download   l) List    u) Update   c) Config   h) Help   q) Quit
---------------------------------------------------------------------------
Downloader> q


True

## C-1 run tutorial
https://www.datacamp.com/community/tutorials/stemming-lemmatization-python

In [20]:
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer

In [21]:
#create an object of class PorterStemmer
porter = PorterStemmer() # PorterStemmer is known for its simplicity and speed
lancaster=LancasterStemmer()
#proide a word to be stemmed
print("Porter Stemmer:")
print(porter.stem("cats"))
print(porter.stem("trouble"))
print(porter.stem("troubling"))
print(porter.stem("troubled"))
print("Lancaster Stemmer:")
print(lancaster.stem("cats"))
print(lancaster.stem("trouble"))
print(lancaster.stem("troubling"))
print(lancaster.stem("troubled"))

Porter Stemmer
cat
troubl
troubl
troubl
Lancaster Stemmer
cat
troubl
troubl
troubl


In [23]:
#A list of words to be stemmed

'''
PorterStemmer is known for its simplicity and speed. 
It is commonly useful in Information Retrieval Environments known as IR Environments 
for fast recall and fetching of search queries.
'''
word_list = ["friend", "friendship", "friends", "friendships","stabil","destabilize","misunderstanding","railroad","moonlight","football"]
print("{0:20}{1:20}{2:20}".format("Original Word:","Porter Stemmer:","lancaster Stemmer:"))
for word in word_list:
    print("{0:20}{1:20}{2:20}".format(word,porter.stem(word),lancaster.stem(word)))

Original Word:      Porter Stemmer:     lancaster Stemmer:  
friend              friend              friend              
friendship          friendship          friend              
friends             friend              friend              
friendships         friendship          friend              
stabil              stabil              stabl               
destabilize         destabil            dest                
misunderstanding    misunderstand       misunderstand       
railroad            railroad            railroad            
moonlight           moonlight           moonlight           
football            footbal             footbal             


In [25]:
# stem sentences
sentence="Pythoners are very intelligent and work very pythonly and now they are pythoning their way to success."
porter.stem(sentence)

'pythoners are very intelligent and work very pythonly and now they are pythoning their way to success.'

In [26]:
from nltk.tokenize import sent_tokenize, word_tokenize
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    for word in token_words:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

x=stemSentence(sentence)
print(x)

python are veri intellig and work veri pythonli and now they are python their way to success . 


## C-main run stemming, create a map between stemmed and original words

In [83]:
print(len(bag_of_words_dict_1)) # use bag_of_words_dict_1 here

10069


In [88]:
from nltk.stem import PorterStemmer

porter = PorterStemmer() # PorterStemmer is known for its simplicity and speed
bag_of_words_dict_2 = dict() # save value of bag_of_words_dict_1 as of key bag_of_words_dict_2

for key,value in bag_of_words_dict_1.items():
    word = value
    word_stem = porter.stem(word)
    
    if word_stem not in list(bag_of_words_dict_2.keys()): # cannot find stem in key list
        bag_of_words_dict_2.update({ word_stem:[word] })
    else: # find stem in key list
        bag_of_words_dict_2[word_stem].append(word)
    
    if key != value: # for example, key = huuuuge, value = huge, then add hugh & huuuuge to dict
        bag_of_words_dict_2[word_stem].append(key)

In [89]:
bag_of_words_dict_2

{'and': ['and', 'aaand', 'anded', 'anding', 'ands'],
 'abandon': ['abandon', 'abandoned', 'abandonment'],
 'abc': ['abc'],
 'abe': ['abe'],
 'abi': ['abi'],
 'abil': ['ability'],
 'abl': ['able'],
 'aboard': ['aboard'],
 'abolit': ['abolition'],
 'abra': ['abra'],
 'abroad': ['abroad'],
 'abruptli': ['abruptly'],
 'absent': ['absent'],
 'absolut': ['absolute', 'absolutely'],
 'abstract': ['abstract', 'abstracted', 'abstraction'],
 'absurd': ['absurd', 'absurdity'],
 'abt': ['abt'],
 'abus': ['abusing', 'abusive'],
 'abuzz': ['abuzz'],
 'academ': ['academic', 'academics'],
 'academi': ['academy'],
 'acceler': ['accelerate', 'accelerates', 'accelerating', 'accelerator'],
 'accept': ['accept',
  'acceptable',
  'acceptance',
  'accepted',
  'accepting',
  'accepts'],
 'access': ['access', 'accessed', 'accessibility', 'accessible', 'accessing'],
 'accident': ['accidentally'],
 'accid': ['accidents'],
 'acclaim': ['acclaim'],
 'accommod': ['accommodate'],
 'accompani': ['accompanying'],
 'a

In [97]:
print(len(bag_of_words_dict_2))

6950


In [94]:
def save_dict_to_file(diction,path):
    f = open(path + 'Zhiyuan_Li_word_dict_after_stemming.csv','w+')
    for key,value in diction.items():
        line = ','.join(str(x) for x in value)
        f.write(key + ':' + line + '\n')
    f.close()
    return None

In [98]:
save_dict_to_file(diction=bag_of_words_dict_2,path=path)

In [99]:
path

'/mnt/c/Users/Administrator.DESKTOP-OHBO1TS/Desktop/Prof_Eesley/github/'